# Binomial Option Pricing


## Parameters

In [4]:
import math

# Option Parameters
S0 = 100.  # initial index level
K = 105.  # strike price
T = 1.  # call option maturity
r = 0.05  # constant short rate
vola = 0.20  # constant volatility factor of diffusion

# Time Parameters
M = 1000  # time steps
dt = T / M  # length of time interval
df = math.exp(-r * dt)  # discount factor per time interval

# Binomial Parameters
u = math.exp(vola * math.sqrt(dt))  # up-movement
d = 1 / u  # down-movement
q = (math.exp(r * dt) - d) / (u - d)  # martingale probability

## With Python Loops

In [9]:
import numpy as np
def oVal_py():
    # LOOP 1 - Index Levels
    S = np.zeros((M + 1, M + 1), dtype=np.float64)  # index level array
    S[0, 0] = S0
    z1 = 0
    for j in range(1, M + 1, 1):
        z1 += 1
        for i in range(z1 + 1):
            S[i, j] = S[0, 0] * (u ** j) * (d ** (i * 2)) 
    # LOOP 2 - Inner Values
    iv = np.zeros((M + 1, M + 1), dtype=np.float64)  # inner value array
    z2 = 0
    for j in range(0, M + 1, 1):
        for i in range(z2 + 1):
            iv[i, j] = max(S[i, j] - K, 0)
        z2 += 1
    # LOOP 3 - Valuation
    pv = np.zeros((M + 1, M + 1), dtype=np.float64)  # present value array
    pv[:, M] = iv[:, M]  # initialize last time point
    z3 = M + 1
    for j in range(M - 1, -1, -1):
        z3 = z3 - 1
        for i in range(z3):
            pv[i, j] = (q * pv[i, j + 1] + (1 - q) * pv[i + 1, j + 1]) * df
    return pv[0, 0]

In [10]:
%time C = oVal_py()

CPU times: user 840 ms, sys: 6.56 ms, total: 846 ms
Wall time: 846 ms


In [11]:
round(C, 3)

8.021

## With NumPy

In [12]:

def oVal_np():
    # Index Levels with NumPy
    mu = np.arange(M + 1)
    mu = np.resize(mu, (M + 1, M + 1))
    md = np.transpose(mu)
    mu = u ** (mu - md)
    md = d ** md
    S = S0 * mu * md
    # Valuation Loop
    V = np.maximum(S - K, 0)
    Qu = np.zeros((M + 1, M + 1), dtype=np.float64)
    Qu[:, :] = q  
    Qd = 1 - Qu 
    z = 0
    for t in range(M - 1, -1, -1):  # backwards iteration
        V[0:M - z, t] = (Qu[0:M - z, t] * V[0:M - z, t + 1]
                        + Qd[0:M - z, t] * V[1:M - z + 1, t + 1]) * df
        z += 1
    return V[0, 0]

In [13]:
%time C = oVal_np()
round(C, 3)

CPU times: user 87.1 ms, sys: 17.2 ms, total: 104 ms
Wall time: 104 ms


8.021